In [13]:
###MNIST 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split 

import torch 
import torch.nn as nn 
from torchvision import transforms,models 

from torch.autograd import Variable 
import torch.optim as optim 
import torch.nn.functional as F 
import os 
path='./data/mnist/'
print(path)

./data/mnist/


In [14]:
train=pd.read_csv(os.path.join(path+'train.csv'),dtype=np.float32)
final_test=pd.read_csv(os.path.join(path+'test.csv'),dtype=np.float32)
train.label.head()

0    1.0
1    0.0
2    1.0
3    4.0
4    0.0
Name: label, dtype: float32

In [15]:
print(train.head())
print(train.describe())
print(train.info())


   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0    1.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1    0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2    1.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3    4.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4    0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0     0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
1     0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
2     0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
3     0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
4     0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   

   pixel780  pixel781  pixel782  pixel783  
0       0.0       0.0       0.

In [16]:
target_np=train.label.values 
feature_np=train.loc[:,train.columns!='lable'].values/255
features_train,features_test,target_train,target_test=train_test_split(feature_np,target_np,test_size=0.2,random_state=42)


In [17]:
!conda install pytorch torchvision torchaudio cudatoolkit=11.0 -c pytorch
device=torch.device('cuda'if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')